In [2]:
pip install jsonlines

In [3]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import jsonlines
import re
from nltk.corpus import stopwords
from nltk.stem import ISRIStemmer

In [5]:
def read_jsonl_to_dataframe(file_path):
    data = []
    with jsonlines.open(file_path) as reader:
        for line in reader:
            data.append(line)
    df = pd.DataFrame(data)
    return df

In [6]:
test_df = read_jsonl_to_dataframe('/kaggle/input/nlpara/arabic_test.jsonl')
val_df = read_jsonl_to_dataframe('/kaggle/input/nlpara/arabic_val.jsonl')
train_df = read_jsonl_to_dataframe('/kaggle/input/nlpara/arabic_train.jsonl')

In [7]:
train_df.head()

,id,url,title,summary,text
0,140323_russian_troops_crimea_naval_base,https://www.bbc.com/arabic/worldnews/2014/03/1...,القوات الأوكرانية تبدأ الانسحاب من القرم,بدأت القوات الأوكرانية الانسحاب من شبه جزيرة ا...,وكان الرئيس الأوكراني المؤقت، الكسندر تورتشينو...
1,130528_egypt_nile_dam,https://www.bbc.com/arabic/middleeast/2013/05/...,هل يفرض سد النهضة الإثيوبي واقعا جديدا على مصر؟,"""هل سيتم تغيير العبارة الشهيرة للمؤرخ اليوناني...",بحلول عام 2050 ستحتاج مصر إلى 21 مليار متر مكع...
2,world-47242349,https://www.bbc.com/arabic/world-47242349,تعرف على منطقة كشمير التي تسببت بحربين بين اله...,قالت الشرطة في القطاع الهندي من إقليم كشمير إن...,وذكرت وكالة الأنباء المحلية (جي.إن.إس) أن جماع...
3,vert-cul-55078328,https://www.bbc.com/arabic/vert-cul-55078328,ماذا تعرف عن العالم الخفي للمعابد اليابانية ال...,في عام 816، تجول راهب يدعى كوكاي، في المنحدرات...,ووقع اختياره على واد عمقه 800 متر محاط بثماني ...
4,141023_yemen_hodeida,https://www.bbc.com/arabic/middleeast/2014/10/...,"اشتباك بين الحوثيين و""الحراك التهامي"" في الحدي...","أكد مصدر في ""الحراك التهامي"" لأبناء محافظة الح...",مسلح حوثي في إب وقال المصدر إن المسلحين الحوثي...


In [8]:
test_df.head()

,id,url,title,summary,text
0,130806_nidhal_hassan_trial,https://www.bbc.com/arabic/worldnews/2013/08/1...,نضال حسن يمثل أمام محكمة عسكرية أمريكية,تنظر محكمة عسكرية أمريكية في وقت لاحق من اليوم...,نضال حسن واعترف نضال حسن، الذي يدافع عن نفسه، ...
1,160129_germany_asylum_seekers,https://www.bbc.com/arabic/worldnews/2016/01/1...,ألمانيا تسعى للحد من الهجرة بإعلان الجزائر وتو...,كشفت ألمانيا النقاب عن خطط لإضافة الجزائر والم...,ألمانيا تواجه مصاعب في التعامل مع الأعداد المت...
2,130729_syria_homs_area_retaken,https://www.bbc.com/arabic/middleeast/2013/07/...,الخالدية : التليفزيون السوري يعلن استعادة الجي...,قال التليفزيون السوري إن قوات الحكومة استعادت ...,"وأكدت وسائل الإعلام السورية أن الجيش ""استعاد ا..."
3,140517_arsenal_fa_cup_winners,https://www.bbc.com/arabic/sports/2014/05/1405...,الارسنال يتوج بلقب كأس انكلترا,توج فريق الارسنال ببطولة كأس انجلترا لكرة القد...,وفاجأ هال سيتي الحضور بمباغتة الارسنال بهدفين ...
4,140722_iraq_minorities,https://www.bbc.com/arabic/middleeast/2014/07/...,العراق: الأقليات في سهل نينوى,يوضع الصراع في العراق غالبا في إطار صراع بين ا...,معبد يزيدي في سهول محافظة نينوى يعيش المسيحيون...


In [9]:
val_df.head()

,id,url,title,summary,text
0,140921_yemen_pm_resign,https://www.bbc.com/arabic/middleeast/2014/09/...,اتفاق لتشكيل حكومة جديدة بين الحوثيين وعدة احز...,وقع الحوثيون اتفاقا مع عدد من الاحزاب اليمنية ...,انضم بعض عناصر القوات المسلحة الى الحوثيين في ...
1,inthepress-56197077,https://www.bbc.com/arabic/inthepress-56197077,مظاهرات تونس: إلى أي مدى سيصل الصراع بين أطراف...,ناقش معلقون في صحف عربية تطور الأزمة السياسية ...,الرئيس التونسي قيس سعيد ورئيس الحكومة هشام الم...
2,151114_gatwick_airport_frenchman_charged,https://www.bbc.com/arabic/worldnews/2015/11/1...,اتهام رجل بحيازة أسلحة بعد إعلان حالة تأهب في ...,اتهمت الشرطة البريطانية رسميا رجلا فرنسيا بحيا...,المسافرون انتظروا ست ساعات على أرض مطار غاتويك...
3,media-48880286,https://www.bbc.com/arabic/media-48880286,عالم سري للسوريّات على فيسبوك محظور على الرجال...,تطلب شابة على إحدى صفحات فيسبوك المغلقة نصيحة ...,وانهالت تعليقات النساء على هذه المداخلة بين مؤ...
4,130311_us_nkorea,https://www.bbc.com/arabic/middleeast/2013/03/...,الولايات المتحدة توسع العقوبات ضد كوريا الشمالية,فرضت الولايات المتحدة عقوبات على بنك العملة ال...,قطعت كوريا الشمالية الخط الهاتفي الساخن بين ال...


In [10]:
train_df=train_df.drop(['id', 'url', 'title'], axis=1)
test_df=test_df.drop(['id', 'url', 'title'], axis=1)
val_df=val_df.drop(['id', 'url', 'title'], axis=1)

In [11]:
train_df.head()

,summary,text
0,بدأت القوات الأوكرانية الانسحاب من شبه جزيرة ا...,وكان الرئيس الأوكراني المؤقت، الكسندر تورتشينو...
1,"""هل سيتم تغيير العبارة الشهيرة للمؤرخ اليوناني...",بحلول عام 2050 ستحتاج مصر إلى 21 مليار متر مكع...
2,قالت الشرطة في القطاع الهندي من إقليم كشمير إن...,وذكرت وكالة الأنباء المحلية (جي.إن.إس) أن جماع...
3,في عام 816، تجول راهب يدعى كوكاي، في المنحدرات...,ووقع اختياره على واد عمقه 800 متر محاط بثماني ...
4,"أكد مصدر في ""الحراك التهامي"" لأبناء محافظة الح...",مسلح حوثي في إب وقال المصدر إن المسلحين الحوثي...


In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
stop_words = set(stopwords.words('arabic'))
stemmer = ISRIStemmer()

In [14]:
def clean_text_arabic(text):
    # Remove unwanted characters and symbols
    text = re.sub(r"[^\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\uFB50-\uFBC2\uFBD3-\uFDFF\uFE70-\uFEFE]", " ", text)
    # Tokenization
    tokens = text.split()
    # Remove stop words
    tokens = [token for token in tokens if token not in stop_words]
    # Stemming
    tokens = [stemmer.stem(token) for token in tokens]
    # Join tokens back into a single string
    cleaned_text = ' '.join(tokens)
    return cleaned_text

In [15]:
def clean_text_data(df):
    df['summary'] = df['summary'].apply(clean_text_arabic)
    df['text'] = df['text'].apply(clean_text_arabic)
    return df

In [16]:
traindf = clean_text_data(train_df)
testdf = clean_text_data(test_df)
valdf = clean_text_data(val_df)

In [17]:
traindf.head()

,summary,text
0,بدأ قوت اوكرانية سحب جزر قرم,وكان رئس وكر ؤقت، كسندر تورتشينوف، امر سحب قوت...
1,يتم غير عبر شهر ؤرخ يون هيرودو مصر هبة نيل ، س...,حلل عام حاج مصر لير متر كعب حصت حلي حسب خبراء ...
2,قلت شرط قطع هند قلم كشمير فجر قنبل ادى قتل عنص...,ذكر وكل باء حلة جي اس جمع جيش حمد تشدد اعل سؤل...
3,عام ، تجل رهب دعى كوكاي، حدر كثف شجر جبل كوا ك...,وقع خير واد عمق متر حاط بثم قمم حدة تشب ورق زه...
4,اكد صدر حرك همي أبناء حفظ حدد لبي سي قتل سلح ح...,سلح حوث اب وقل صدر سلح حوث هجم شطء حرك همي إفش...


In [18]:
def identify_outliers(df, column, lower_threshold=10, upper_threshold=1000):
    lengths = df[column].apply(lambda x: len(x.split()))
    outliers = df[(lengths < lower_threshold) | (lengths > upper_threshold)]
    return outliers

In [19]:
# Remove outliers from the DataFrame
def remove_outliers(df, column, lower_threshold=10, upper_threshold=1000):
    lengths = df[column].apply(lambda x: len(x.split()))
    filtered_df = df[(lengths >= lower_threshold) & (lengths <= upper_threshold)]
    return filtered_df

In [20]:
# Specify the lower and upper thresholds for text length
lower_threshold = 10
upper_threshold = 1000

In [21]:
outliers = identify_outliers(train_df, 'text', lower_threshold, upper_threshold)
train_df_filtered = remove_outliers(train_df, 'text', lower_threshold, upper_threshold)
print(f"Number of outliers removed: {len(outliers)}")

Number of outliers removed: 1122


In [22]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [23]:
from transformers import BartTokenizer, BartForConditionalGeneration
import torch

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [24]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

In [25]:
max_length = max(len(text.split()) for text in train_df_filtered['text'].tolist())
print("Maximum text length:", max_length)

Maximum text length: 1000


In [26]:
train_encodings = tokenizer.batch_encode_plus(
    train_df_filtered['text'].tolist(),
    truncation=True,
    padding='longest',
    max_length=1000,
    return_tensors='pt'
)

In [27]:
max_length = max(len(text.split()) for text in train_df_filtered['summary'].tolist())
print("Maximum text length:", max_length)

Maximum text length: 187


In [28]:
train_labels = tokenizer.batch_encode_plus(
    train_df_filtered['summary'].tolist(),
    truncation=True,
    padding='longest',
    max_length=187,
    return_tensors='pt'
)

In [29]:
# PyTorch dataset
train_dataset = torch.utils.data.TensorDataset(
    train_encodings['input_ids'],
    train_encodings['attention_mask'],
    train_labels['input_ids'],
    train_labels['attention_mask']
)

In [30]:
# Load the BART model for sequence-to-sequence conditional generation
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')


In [31]:
# Fine-tuning BART 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [32]:
# Define the optimizer and the number of training epochs
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
num_epochs = 5

In [33]:
# Train the model
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)

In [34]:
for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_loader:
        input_ids, attention_mask, labels, labels_attention_mask = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        labels_attention_mask = labels_attention_mask.to(device)

        optimizer.zero_grad()
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=labels[:, :-1].contiguous(),
            labels=labels[:, 1:].contiguous(),
            decoder_attention_mask=labels_attention_mask[:, :-1].contiguous(),
        )
        loss = outputs.loss
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {total_loss / len(train_loader)}")

Epoch 1/5 - Loss: 0.9397075096034742
Epoch 2/5 - Loss: 0.746882768879225
Epoch 3/5 - Loss: 0.7038230224657844
Epoch 4/5 - Loss: 0.6771295473634542
Epoch 5/5 - Loss: 0.6582427135863147


In [35]:
model.save_pretrained('./saved_models/bart_fine_tuned')

In [36]:
# Load the fine-tuned BART model for inference
model = BartForConditionalGeneration.from_pretrained('./saved_models/bart_fine_tuned')

In [37]:
# Set the model to evaluation mode
model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [38]:
torch.cuda.is_available()

True

In [39]:
torch.version.cuda

'11.8'

In [40]:
torch.cuda.set_device(0)

In [41]:
pip install --upgrade torch transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 82.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 58.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 72.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 8.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 11.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 5.8 MB/s eta 0:00:0000:0100:01
     ━━━━━